In [ ]:
import mysql.connector
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[6]").appName('reddit').getOrCreate()

comments = spark.read.format("csv")\
                     .option("sep", "\t")\
                     .options(inferschema='true',header='true')\
                     .load('/home/ubuntu/cs179g_venv/src/comments_join.tsv')
comments_df = comments.toPandas()

In [ ]:
# remove 'Title' column and fill NaNs with 'N/A' to prevent null error later
comments_df.rename({'Title' : 'Title', 'Combo': 'Comments'}, axis=1, inplace=True)
comments_df.Comments = comments_df.Comments.fillna('N/A')
comments_df = comments_df.drop('Title', axis=1)

In [ ]:
db_connection = mysql.connector.connect(user="mko022", password="password_daniel")
db_cursor = db_connection.cursor()
db_cursor.execute("USE cs179g;")

db_cursor.execute("CREATE TABLE comments(Title VARCHAR(510) NOT NULL, Comments VARCHAR(800) NOT NULL);")

posts_tuples = list(comments_df.itertuples(index=False, name=None))
posts_string = str(posts_tuples).strip('[]')

db_cursor.execute("INSERT INTO comments(Title, Comments) VALUES " + posts_string + ";")
db_cursor.execute("FLUSH TABLES;")

# check if it works
db_cursor.execute("SELECT * FROM comments LIMIT 10;")
print(db_cursor.fetchall())